In [ ]:
!pip install --upgrade datasets fsspec
# Seems to prevent the `Invalid pattern: '**' can only be an entire path component` error
# Make sure to manually restart the session

!pip install lime

In [ ]:
import os

current_directory = os.getcwd()
print("Current working directory:", current_directory)

In [ ]:
import sys
print("Jupyter's Python Executable:", sys.executable)
print("Jupyter's sys.path:", sys.path)

In [ ]:
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoConfig,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    AutoModelForSequenceClassification
)
from peft import LoraConfig, PeftModel, get_peft_model
from torch import nn
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, f1_score, classification_report, multilabel_confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')
from lime.lime_text import LimeTextExplainer

# constants
english_pretrained_model_path = './models/zero_shot'
emotion_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
MODEL_NAME = "Davlan/afro-xlmr-base"
MAX_LENGTH = 128

class MultiLabelEmotionModel(nn.Module):
    """Multilabel emotion classification model based on paraphrase-xlm-r-multilingual-v1"""

    def __init__(self, model_name: str, num_labels: int, dropout_rate: float = 0.3):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.backbone = AutoModel.from_pretrained(model_name)
        self.num_labels = num_labels

        # Add classification head for multilabel prediction
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)

        # Use [CLS] token representation
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            # Use BCEWithLogitsLoss for multilabel classification
            loss_fn = nn.BCEWithLogitsLoss()
            loss = loss_fn(logits, labels.float())

        return {"loss": loss, "logits": logits}

class BrighterMultiLabelDataset(Dataset):
    """Custom dataset for BRIGHTER multilabel emotion data"""

    def __init__(self, texts: List[str], labels: np.ndarray, tokenizer, max_length: int = 512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        labels = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

def load_and_preprocess_data(lang):
    """Load and preprocess the BRIGHTER dataset for multilabel classification"""
    print("Loading BRIGHTER dataset...")

    # Load the dataset
    dataset = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", lang)

    train_df = pd.DataFrame(dataset['train'])
    test_df = pd.DataFrame(dataset['test'])

    print(f"Train samples: {len(train_df)}")
    print(f"Test samples: {len(test_df)}")

    emotion_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

    print(f"Emotion columns: {emotion_columns}")

    # Check if all emotion columns exist
    missing_cols = [col for col in emotion_columns if col not in train_df.columns]
    if missing_cols:
        print(f"Warning: Missing columns {missing_cols}")
        print(f"Available columns: {list(train_df.columns)}")
        emotion_cols = [col for col in train_df.columns if any(emotion in col.lower() for emotion in ['anger', 'disgust', 'fear', 'joy', 'sad', 'surprise', 'happy'])]
        print(f"Potential emotion columns: {emotion_cols}")
        emotion_columns = emotion_cols[:6]  # Take first 6 emotion columns

    # Fill None values in emotion columns with 0
    # This handles cases where an emotion might not be present and represented as `null`
    train_df[emotion_columns] = train_df[emotion_columns].fillna(0)
    test_df[emotion_columns] = test_df[emotion_columns].fillna(0)

    # Explore emotion distribution
    print("\nEmotion distribution in training set:")
    for emotion in emotion_columns:
        if emotion in train_df.columns:
            count = train_df[emotion].sum()
            percentage = (count / len(train_df)) * 100
            print(f"{emotion}: {count} ({percentage:.1f}%)")

    train_labels = train_df[emotion_columns].values
    test_labels = test_df[emotion_columns].values

    # Calculate statistics
    labels_per_sample = train_labels.sum(axis=1)
    print(f"\nLabel statistics:")
    print(f"Average labels per sample: {labels_per_sample.mean():.2f}")
    print(f"Max labels per sample: {labels_per_sample.max()}")
    print(f"Samples with no labels: {(labels_per_sample == 0).sum()}")
    print(f"Samples with multiple labels: {(labels_per_sample > 1).sum()}")

    return train_df, test_df, emotion_columns

def compute_multilabel_metrics(eval_pred):
    """Compute metrics for multilabel evaluation"""
    predictions, labels = eval_pred

    # Apply sigmoid to get probabilities
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions)).numpy()

    # Convert probabilities to binary predictions (threshold = 0.5)
    binary_predictions = (probs > 0.5).astype(int)

    # Calculate metrics
    # Exact match accuracy (all labels must match)
    exact_match = accuracy_score(labels, binary_predictions)

    # Hamming score - average across all labels
    hamming_score = (binary_predictions == labels).mean()

    # F1 scores
    f1_micro = f1_score(labels, binary_predictions, average='micro')
    f1_macro = f1_score(labels, binary_predictions, average='macro')
    f1_weighted = f1_score(labels, binary_predictions, average='weighted')

    # Per-label accuracy
    per_label_acc = []
    for i in range(labels.shape[1]):
        acc = accuracy_score(labels[:, i], binary_predictions[:, i])
        per_label_acc.append(acc)

    return {
        'exact_match_accuracy': exact_match,
        'hamming_score': hamming_score,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'mean_per_label_accuracy': np.mean(per_label_acc)
    }

def create_data_loaders(train_df, test_df, emotion_columns, tokenizer, max_length=512):
    """Create train and test datasets for multilabel classification"""

    train_dataset = BrighterMultiLabelDataset(
        texts=train_df['text'].tolist(),
        labels=train_df[emotion_columns].values,
        tokenizer=tokenizer,
        max_length=max_length
    )

    test_dataset = BrighterMultiLabelDataset(
        texts=test_df['text'].tolist(),
        labels=test_df[emotion_columns].values,
        tokenizer=tokenizer,
        max_length=max_length
    )

    return train_dataset, test_dataset

def plot_training_history(trainer):
    """Plot training metrics"""
    # Extract training history
    train_logs = [log for log in trainer.state.log_history if 'train_loss' in log]
    eval_logs = [log for log in trainer.state.log_history if 'eval_loss' in log]

    train_steps = [log['step'] for log in train_logs]
    train_loss = [log['train_loss'] for log in train_logs]

    eval_steps = [log['step'] for log in eval_logs]
    eval_loss = [log['eval_loss'] for log in eval_logs]
    eval_f1_macro = [log['eval_f1_macro'] for log in eval_logs]
    hamming_scores = [log['eval_hamming_score'] for log in eval_logs]

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

    # Plot loss
    ax1.plot(train_steps, train_loss, label='Training Loss', color='blue')
    ax1.plot(eval_steps, eval_loss, label='Validation Loss', color='red')
    ax1.set_xlabel('Steps')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    ax1.grid(True)

    # Plot F1 macro
    ax2.plot(eval_steps, eval_f1_macro, label='F1 Macro', color='green')
    ax2.set_xlabel('Steps')
    ax2.set_ylabel('F1 Score')
    ax2.set_title('Validation F1 Macro Score')
    ax2.legend()
    ax2.grid(True)

    # Plot Hamming Score
    ax3.plot(eval_steps, hamming_scores, label='Hamming Score', color='purple')
    ax3.set_xlabel('Steps')
    ax3.set_ylabel('Hamming Score')
    ax3.set_title('Validation Hamming Score')
    ax3.legend()
    ax3.grid(True)

    # Plot both metrics together
    ax4.plot(eval_steps, eval_f1_macro, label='F1 Macro', color='green')
    ax4.plot(eval_steps, hamming_scores, label='Hamming Score', color='purple')
    ax4.set_xlabel('Steps')
    ax4.set_ylabel('Score')
    ax4.set_title('Combined Validation Metrics')
    ax4.legend()
    ax4.grid(True)

    plt.tight_layout()
    plt.show()

def evaluate_model_detailed(model, test_dataset, emotion_columns, tokenizer):
    """Detailed evaluation of the multilabel model"""
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    all_predictions = []
    all_labels = []
    all_probabilities = []

    with torch.no_grad():
        for i in range(len(test_dataset)):
            item = test_dataset[i]
            input_ids = item['input_ids'].unsqueeze(0).to(device)
            attention_mask = item['attention_mask'].unsqueeze(0).to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Apply sigmoid to get probabilities
            probabilities = torch.sigmoid(outputs['logits']).cpu().numpy()[0]
            binary_predictions = (probabilities > 0.5).astype(int)

            all_predictions.append(binary_predictions)
            all_probabilities.append(probabilities)
            all_labels.append(item['labels'].numpy())

    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    all_probabilities = np.array(all_probabilities)

    # Calculate comprehensive metrics
    exact_match = accuracy_score(all_labels, all_predictions)
    hamming_score = (all_predictions == all_labels).mean()
    f1_micro = f1_score(all_labels, all_predictions, average='micro',zero_division=0)
    f1_macro = f1_score(all_labels, all_predictions, average='macro',zero_division=0)
    f1_weighted = f1_score(all_labels, all_predictions, average='weighted',zero_division=0)

    print(f"\n=== Final Test Results ===")
    print(f"Exact Match Accuracy: {exact_match:.4f}")
    print(f"Hamming Score: {hamming_score:.4f}")
    print(f"F1-Score (Micro): {f1_micro:.4f}")
    print(f"F1-Score (Macro): {f1_macro:.4f}")
    print(f"F1-Score (Weighted): {f1_weighted:.4f}")

    # Per-emotion metrics
    print(f"\n=== Per-Emotion Results ===")
    for i, emotion in enumerate(emotion_columns):
        emotion_f1 = f1_score(all_labels[:, i], all_predictions[:, i], zero_division=0)
        emotion_acc = accuracy_score(all_labels[:, i], all_predictions[:, i])
        print(f"{emotion:>8}: F1={emotion_f1:.3f}, Acc={emotion_acc:.3f}")

    # Classification report for each emotion
    print(f"\n=== Detailed Classification Report ===")
    for i, emotion in enumerate(emotion_columns):
        print(f"\n{emotion.upper()}:")

        # Explicitly specify labels=[0, 1] to handle cases with only one class present
        # Also add zero_division handling to classification_report
        print(classification_report(all_labels[:, i], all_predictions[:, i],
                                  target_names=['Not Present', 'Present'], digits=3, labels=[0, 1], zero_division=0))

    return all_predictions, all_labels, all_probabilities

def plot_emotion_distribution(train_df, test_df, emotion_columns):
    """Plot emotion distribution across train and test sets"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Training set
    train_counts = [train_df[col].sum() for col in emotion_columns]
    train_percentages = [count/len(train_df)*100 for count in train_counts]

    ax1.bar(emotion_columns, train_percentages, color='skyblue', alpha=0.7)
    ax1.set_title('Emotion Distribution - Training Set')
    ax1.set_ylabel('Percentage of samples')
    ax1.tick_params(axis='x', rotation=45)

    # Test set
    test_counts = [test_df[col].sum() for col in emotion_columns]
    test_percentages = [count/len(test_df)*100 for count in test_counts]

    ax2.bar(emotion_columns, test_percentages, color='lightcoral', alpha=0.7)
    ax2.set_title('Emotion Distribution - Test Set')
    ax2.set_ylabel('Percentage of samples')
    ax2.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

def train_on_english():
    """Main training pipeline for multilabel emotion classification"""
    BATCH_SIZE = 4
    LEARNING_RATE = 2e-5
    NUM_EPOCHS = 20

    print("=== BRIGHTER Multilabel Emotion Classification Fine-tuning ===\n")

    # Load and preprocess english data
    train_df, test_df, emotion_columns = load_and_preprocess_data("eng")
    num_labels = len(emotion_columns)

    print(f"\nEmotion labels: {emotion_columns}")
    print(f"Number of labels: {num_labels}")

    # Plot emotion distribution
    plot_emotion_distribution(train_df, test_df, emotion_columns)

    if os.path.exists(english_pretrained_model_path):
        print(f"\nLoading fine-tuned model from: {english_pretrained_model_path}")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        base_model = MultiLabelEmotionModel(MODEL_NAME, num_labels)
        peft_model = PeftModel.from_pretrained(base_model, english_pretrained_model_path)
    else:
        # Initialize tokenizer
        print(f"\nLoading tokenizer: {MODEL_NAME}")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

        # Create datasets
        print("Creating datasets...")
        train_dataset, test_dataset = create_data_loaders(train_df, test_df, emotion_columns, tokenizer, MAX_LENGTH)

        # Initialize model
        print(f"Initializing multilabel model with {num_labels} emotion labels...")
        model = MultiLabelEmotionModel(MODEL_NAME, num_labels)

        # LoRA configurations
        lora_config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=["query", "value", "key"], # Changes depending on the model
            lora_dropout=0.1,
            bias="none",
            task_type="SEQ_CLS"
        )

        peft_model = get_peft_model(model, lora_config)
        peft_model.print_trainable_parameters()

        # Training arguments
        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=NUM_EPOCHS,
            per_device_train_batch_size=BATCH_SIZE,
            per_device_eval_batch_size=BATCH_SIZE,
            warmup_steps=100,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=50,
            eval_strategy="steps",
            eval_steps=500,
            save_strategy="steps",
            save_steps=500,
            load_best_model_at_end=True,
            metric_for_best_model="eval_f1_macro",
            greater_is_better=True,
            learning_rate=LEARNING_RATE,
            fp16=torch.cuda.is_available(),
            dataloader_num_workers=2,
            dataloader_pin_memory=True,
            remove_unused_columns=False,
            report_to="none",
        )

        # Initialize trainer
        trainer = Trainer(
            model=peft_model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            compute_metrics=compute_multilabel_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
        )

        # Train the model
        print("CUDA available:", torch.cuda.is_available())
        print("Using device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
        print("\nStarting training...")
        trainer.train()

        # Plot training history
        print("\nPlotting training history...")
        plot_training_history(trainer)

        # Detailed evaluation
        print("\nPerforming detailed evaluation...")
        predictions, labels, probabilities = evaluate_model_detailed(peft_model, test_dataset, emotion_columns, tokenizer)

        # Save the model
        print("\nSaving model...")
        peft_model.save_pretrained('./models/zero_shot')
        tokenizer.save_pretrained('./models/zero_shot')

        # Save emotion columns
        import json
        with open('./models/zero_shot/emotion_columns.json', 'w') as f:
            json.dump(emotion_columns, f)

        print("\nTraining completed! Model saved to './models/zero_shot'")

    #------------------

    return peft_model, tokenizer, emotion_columns

def predict_emotions(text: str, model, tokenizer, emotion_columns, threshold=0.5, device='cpu'):
    """Predict emotions for a single text with probabilities"""
    model.eval()
    model.to(device)

    # Tokenize input
    inputs = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.sigmoid(outputs['logits']).cpu().numpy()[0]

    # Get predictions above threshold
    predictions = probabilities > threshold

    results = []
    for i, (emotion, prob, pred) in enumerate(zip(emotion_columns, probabilities, predictions)):
        results.append({
            'emotion': emotion,
            'probability': prob,
            'predicted': bool(pred)
        })

    # Sort by probability (descending)
    results.sort(key=lambda x: x['probability'], reverse=True)

    return results

In [ ]:
def finetune_on_hausa(pretrained_model, tokenizer, emotion_columns):
    """Phase 2: Fine-tune the English-trained model on Hausa data."""
    BATCH_SIZE = 4
    LEARNING_RATE = 1e-5
    NUM_EPOCHS = 20
    hausa_finetuned_model_path = './models/fine_tuned_hausa'
    output_path = './models/fine_tuned_hausa'

    print("\n=== PHASE 2: Fine-tuning on Hausa Data ===")

     # Load Hausa data
    train_df, test_df, _ = load_and_preprocess_data("hau")
    train_dataset, test_dataset = create_data_loaders(train_df, test_df, emotion_columns, tokenizer, MAX_LENGTH)

    if os.path.exists(hausa_finetuned_model_path):
        num_labels = len(emotion_columns)
        print(f"\nLoading fine-tuned model from: {hausa_finetuned_model_path}")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        base_model = MultiLabelEmotionModel(MODEL_NAME, num_labels)
        pretrained_model = PeftModel.from_pretrained(base_model, hausa_finetuned_model_path)
    else:
        # LoRA config (assumes pretrained_model is already a PEFT model)
      training_args = TrainingArguments(
          output_dir='./results_hausa',
          num_train_epochs=NUM_EPOCHS,
          per_device_train_batch_size=BATCH_SIZE,
          per_device_eval_batch_size=BATCH_SIZE,
          warmup_steps=100,
          weight_decay=0.01,
          logging_dir='./logs_hausa',
          logging_steps=50,
          eval_strategy="steps",
          eval_steps=1000,
          save_strategy="steps",
          save_steps=1000,
          load_best_model_at_end=True,
          metric_for_best_model="eval_f1_macro",
          greater_is_better=True,
          learning_rate=LEARNING_RATE,
          fp16=torch.cuda.is_available(),
          dataloader_num_workers=2,
          dataloader_pin_memory=True,
          remove_unused_columns=False,
          report_to="none",
      )

      trainer = Trainer(
          model=pretrained_model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=test_dataset,
          compute_metrics=compute_multilabel_metrics,
          callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
      )

      print("\nStarting Hausa fine-tuning...")
      trainer.train()

      # Save Hausa fine-tuned model
      pretrained_model.save_pretrained(output_path)
      tokenizer.save_pretrained(output_path)

      print("\nHausa fine-tuned model saved to:", output_path)

      # Detailed evaluation
      print("\nPerforming detailed evaluation...")
      predictions, labels, probabilities = evaluate_model_detailed(pretrained_model, test_dataset, emotion_columns, tokenizer)

    return pretrained_model

In [ ]:
class LimeMultiLabelEmotionExplainer:
    """
    LIME explainer wrapper for multilabel emotion classification models
    """

    def __init__(self, model, tokenizer, emotion_columns: List[str], device='cuda' if torch.cuda.is_available() else 'cpu'):
        """
        Initialize the LIME explainer

        Args:
            model: Your trained MultiLabelEmotionModel (or PeftModel)
            tokenizer: The tokenizer used for preprocessing
            emotion_columns: List of emotion label names
            device: Device to run inference on
        """
        self.model = model
        self.tokenizer = tokenizer
        self.emotion_columns = emotion_columns
        self.device = device
        self.model.to(device)
        self.model.eval()

        # Initialize LIME explainer
        self.explainer = LimeTextExplainer(
            class_names=emotion_columns
        )

    def predict_proba(self, texts: List[str]) -> np.ndarray:
        """
        Prediction function for LIME - returns probabilities for all emotions

        Args:
            texts: List of text strings to predict

        Returns:
            numpy array of shape (n_samples, n_emotions) with probabilities
        """
        self.model.eval()
        predictions = []

        with torch.no_grad():
            for text in texts:
                # Tokenize the text
                inputs = self.tokenizer(
                    text,
                    return_tensors='pt',
                    truncation=True,
                    padding=True,
                    max_length=512
                ).to(self.device)

                # Get model predictions
                outputs = self.model(**inputs)
                logits = outputs['logits']

                # Convert to probabilities using sigmoid (for multilabel)
                probs = torch.sigmoid(logits).cpu().numpy().flatten()
                predictions.append(probs)

        return np.array(predictions)

    def explain_instance(self, text: str, top_labels: int = None, num_features: int = 10,
                        num_samples: int = 1000) -> Dict:
        """
        Explain a single text instance

        Args:
            text: Text to explain
            top_labels: Number of top emotions to explain (None for all)
            num_features: Number of words to include in explanation
            num_samples: Number of samples for LIME

        Returns:
            Dictionary containing explanations and predictions
        """
        # Get prediction for the original text
        original_pred = self.predict_proba([text])[0]

        # Determine which emotions to explain
        if top_labels is None:
            labels_to_explain = list(range(len(self.emotion_columns)))
        else:
            # Get top predicted emotions
            top_indices = np.argsort(original_pred)[-top_labels:][::-1]
            labels_to_explain = top_indices.tolist()

        # Generate LIME explanation
        explanation = self.explainer.explain_instance(
            text,
            self.predict_proba,
            labels=labels_to_explain,
            num_features=num_features,
            num_samples=num_samples
        )

        # Format results
        results = {
            'text': text,
            'predictions': {emotion: float(prob) for emotion, prob in zip(self.emotion_columns, original_pred)},
            'explanations': {}
        }

        # Extract explanations for each emotion
        for label_idx in labels_to_explain:
            emotion_name = self.emotion_columns[label_idx]
            word_importance = explanation.as_list(label=label_idx)
            results['explanations'][emotion_name] = word_importance

        return results

    def visualize_explanation(self, explanation_result: Dict, emotion: str = None,
                            save_path: str = None, figsize: Tuple[int, int] = (12, 8)):
        """
        Visualize LIME explanations

        Args:
            explanation_result: Result from explain_instance
            emotion: Specific emotion to visualize (None for all)
            save_path: Path to save the plot
            figsize: Figure size
        """
        explanations = explanation_result['explanations']
        predictions = explanation_result['predictions']

        if emotion and emotion in explanations:
            emotions_to_plot = [emotion]
        else:
            emotions_to_plot = list(explanations.keys())

        n_emotions = len(emotions_to_plot)
        fig, axes = plt.subplots(n_emotions, 1, figsize=figsize, squeeze=False)

        for i, emotion_name in enumerate(emotions_to_plot):
            ax = axes[i, 0]
            word_importance = explanations[emotion_name]

            # Separate positive and negative influences
            words = [item[0] for item in word_importance]
            scores = [item[1] for item in word_importance]

            # Create color map based on positive/negative influence
            colors = ['green' if score > 0 else 'red' for score in scores]

            # Create horizontal bar plot
            y_pos = np.arange(len(words))
            bars = ax.barh(y_pos, scores, color=colors, alpha=0.7)

            ax.set_yticks(y_pos)
            ax.set_yticklabels(words)
            ax.set_xlabel('Importance Score')
            ax.set_title(f'{emotion_name} (Prediction: {predictions[emotion_name]:.3f})')
            ax.grid(axis='x', alpha=0.3)

            # Add vertical line at x=0
            ax.axvline(x=0, color='black', linestyle='-', alpha=0.3)

        plt.tight_layout()
        plt.suptitle(f'LIME Explanations\nText: "{explanation_result["text"][:100]}..."',
                     y=1.02, fontsize=12)

        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=300)

        plt.show()

def demo_lime_usage(model, tokenizer, emotion_columns):
    explainer = LimeMultiLabelEmotionExplainer(model, tokenizer, emotion_columns)
    text = "I'm so happy and excited!"
    explanation = explainer.explain_instance(text, top_labels=6)
    explainer.visualize_explanation(explanation)

In [ ]:
# Run the main training pipeline
english_model, tokenizer, emotion_columns = train_on_english()
hausa_model = finetune_on_hausa(english_model, tokenizer, emotion_columns)


print("\n=== Example Multilabel Predictions ===")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load and preprocess hausa data
train_df, test_df, emotion_columns = load_and_preprocess_data("hau")
print(test_df.head(5))

demo_lime_usage(hausa_model, tokenizer, emotion_columns)

# Test zero shot inference
train_dataset, test_dataset = create_data_loaders(train_df, test_df, emotion_columns, tokenizer, MAX_LENGTH)
predictions, labels, probabilities = evaluate_model_detailed(english_model, test_dataset, emotion_columns, tokenizer)

# Example predictions
test_texts = [
    "I am so happy and excited about this wonderful news!",
    "This makes me really angry and disgusted.",
    "This is absolutely baffling.",
    "What a joyful and surprising moment!",
    "I'm feeling quite sad and afraid lately.",
    "I can't believe this happened, I'm shocked and happy at the same time!"
]

for text in test_texts:
    print(f"\nText: '{text}'")
    results = predict_emotions(text, english_model, tokenizer, emotion_columns, threshold=0.3, device=device)

    print("Predicted emotions:")
    for result in results:
        status = "✓" if result['predicted'] else " "
        print(f"  {status} {result['emotion']:>8}: {result['probability']:.3f}")

    predicted_emotions = [r['emotion'] for r in results if r['predicted']]
    print(f"Final prediction: {predicted_emotions if predicted_emotions else 'No emotions detected'}")
    print("-" * 60)

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer
from torch.nn.functional import sigmoid

def predict_proba(texts):
    """Wraps the model for LIME"""
    device='cuda' if torch.cuda.is_available() else 'cpu'
    tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = hausa_model(**tokens)
        logits = outputs['logits'] if isinstance(outputs, dict) else outputs.logits
        probs = sigmoid(logits).cpu().numpy()  # For multilabel use sigmoid
    return probs


In [ ]:
# Define class names
class_names = emotion_columns

# Create the explainer
explainer = LimeTextExplainer(class_names=class_names)

# Choose a sample text
sample_text = "hmmm wai hauka, kamar qasa mai mutum sama da 200m ake maganar wai wannan shine jaririn farko? wllh ba gaskiya bane shidai suka fada kawai🤔"

# Explain prediction
exp = explainer.explain_instance(sample_text, predict_proba, num_features=10, labels=list(range(len(class_names))))

# Show explanation for each label
for label_index in range(len(class_names)):
    print(f"\nExplanation for label: {class_names[label_index]}")
    exp.show_in_notebook(text=sample_text, labels=[label_index])  # Or use exp.as_list(label=label_index)

# Evaluation

In [ ]:
path = "KhweziSandi/XLM-R-Zero-Shot-Hausa"

tokenizer = AutoTokenizer.from_pretrained(path)
base_model = MultiLabelEmotionModel(MODEL_NAME, len(emotion_columns))
model_to_eval = PeftModel.from_pretrained(base_model, path)

train_df, test_df, _ = load_and_preprocess_data("hau")
train_dataset, test_dataset = create_data_loaders(train_df, test_df, emotion_columns, tokenizer, MAX_LENGTH)
predictions, labels, probabilities = evaluate_model_detailed(model_to_eval, test_dataset, emotion_columns, tokenizer)